# Extrinsic calibration & pose estimation using openCV

# 0. Import Dependencies

In [11]:
import numpy as np
import cv2
import cv2 as cv
import glob
import matplotlib.pyplot as plt
%matplotlib inline
('Enviroments Ready!')

'Enviroments Ready!'

# 1. Extrinsic Calibration

In [13]:
# Documentation of the code: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

# checkerboard Dimensions
cbrow = 5
cbcol = 8

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cbrow * cbcol, 3), np.float32)
objp[:, :2] = np.mgrid[0:cbcol, 0:cbrow].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# Path to images taken
images = glob.glob(r'C:\Users\Bruker\Desktop\Master 3.semester\MAS512-Maskinsyn og innebygd AI\Lab_exercises\Lab_06.09\*.png')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (cbcol, cbrow), None)
    
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        
        # Draw and display the corners
        cv.drawChessboardCorners(img, (cbrow,cbcol), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

############## CALIBRATION ##############
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 2. Pose Estimation

In [14]:
# Documentation of the code: https://docs.opencv.org/3.4/d7/d53/tutorial_py_pose.html

# Function to draw colored axis for pose estimation

def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)


 # Find the rotation and translation vectors.
ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
# project 3D points to image plane
imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)

corners2_new = corners2.astype(int)
imgpts_new = imgpts.astype(int)
img = draw(img,corners2_new,imgpts_new)
cv.imshow('img',img)
k = cv.waitKey(0) & 0xFF
if k == ord('s'):
    cv.imwrite(fname[:6]+'.png', img)
cv.destroyAllWindows()